In [ ]:
"""
# Results Analysis

Analyze all experimental results
"""

In [ ]:
# Cell 1: Load results
import sys
sys.path.insert(0, '..')

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

RESULTS_DIR = Path("../results/expt_3")

# Load all results
all_results = []
for pkl_file in RESULTS_DIR.glob("*.pkl"):
    with open(pkl_file, 'rb') as f:
        all_results.append(pickle.load(f))

print(f"Loaded {len(all_results)} experiments")

In [ ]:
# Cell 2: Overall statistics
fidelities = [r['test_fidelity_mean'] for r in all_results]

print(f"Overall Statistics:")
print(f"  Mean Fidelity: {np.mean(fidelities):.4f} ± {np.std(fidelities):.4f}")
print(f"  Best: {np.max(fidelities):.4f}")
print(f"  Worst: {np.min(fidelities):.4f}")
print(f"  Experiments > 0.90: {sum(f > 0.90 for f in fidelities)}/{len(fidelities)}")

In [ ]:
# Cell 3: Group by priority
priorities = {'A': [], 'B': [], 'C': [], 'D': []}
for r in all_results:
    if 'PriorityA' in r['config']['name']:
        priorities['A'].append(r)
    elif 'PriorityB' in r['config']['name']:
        priorities['B'].append(r)
    elif 'PriorityC' in r['config']['name']:
        priorities['C'].append(r)
    elif 'PriorityD' in r['config']['name']:
        priorities['D'].append(r)

for priority, results in priorities.items():
    fids = [r['test_fidelity_mean'] for r in results]
    print(f"\nPriority {priority}: {len(results)} experiments")
    print(f"  Mean: {np.mean(fids):.4f} ± {np.std(fids):.4f}")
    print(f"  Range: [{np.min(fids):.4f}, {np.max(fids):.4f}]")

In [ ]:
# Cell 4: Analyze by measurement type
by_measurement = {}
for r in all_results:
    meas = r['config']['measurement_type']
    by_measurement.setdefault(meas, []).append(r['test_fidelity_mean'])

print("\nBy Measurement Type:")
for meas, fids in sorted(by_measurement.items()):
    print(f"  {meas:10s}: {np.mean(fids):.4f} ± {np.std(fids):.4f} (n={len(fids)})")

In [ ]:
# Cell 5: Priority A - Shots vs Performance
priority_a = priorities['A']

shots_data = {}
for r in priority_a:
    if 'shots' in r['config']:
        key = (r['config']['shots'], r['config']['noise_level'], r['config']['measurement_type'])
        shots_data.setdefault(key, []).append(r['test_fidelity_mean'])

# Create plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for noise in [0.0, 0.01, 0.05]:
    shots = []
    baseline_fids = []
    xz_fids = []
    
    for (s, n, m), fids in shots_data.items():
        if n == noise:
            if m == 'baseline':
                shots.append(s)
                baseline_fids.append(np.mean(fids))
            elif m == 'XZ':
                xz_fids.append(np.mean(fids))
    
    if shots:
        axes[0].plot(shots, baseline_fids, 'o-', label=f'Baseline, {int(noise*100)}% noise')
        axes[1].plot(shots, xz_fids, 's-', label=f'XZ, {int(noise*100)}% noise')

axes[0].set_xlabel('Shots')
axes[0].set_ylabel('Mean Fidelity')
axes[0].set_xscale('log')
axes[0].set_title('Baseline Performance')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Shots')
axes[1].set_ylabel('Mean Fidelity')
axes[1].set_xscale('log')
axes[1].set_title('XZ Two-Basis Performance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('shots_analysis.png', dpi=150)
plt.show()

In [ ]:
# Cell 6: Priority B - Ensemble analysis
priority_b = priorities['B']

ensemble_data = {}
for r in priority_b:
    ensemble = r['config']['ensemble_type']
    mixing = r['config'].get('mixing_p')
    key = f"{ensemble}_p{mixing}" if mixing else ensemble
    ensemble_data.setdefault(key, []).append(r['test_fidelity_mean'])

print("\nEnsemble Performance:")
for ensemble, fids in sorted(ensemble_data.items()):
    print(f"  {ensemble:20s}: {np.mean(fids):.4f} ± {np.std(fids):.4f}")

In [ ]:
# Cell 7: Best and worst experiments
best_exp = max(all_results, key=lambda r: r['test_fidelity_mean'])
worst_exp = min(all_results, key=lambda r: r['test_fidelity_mean'])

print("\nBest Experiment:")
print(f"  Name: {best_exp['config']['name']}")
print(f"  Fidelity: {best_exp['test_fidelity_mean']:.4f}")

print("\nWorst Experiment:")
print(f"  Name: {worst_exp['config']['name']}")
print(f"  Fidelity: {worst_exp['test_fidelity_mean']:.4f}")